# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="https://github.com/xlnwel/deep-learning/blob/master/intro-to-rnns/assets/charseq.jpeg?raw=1" width="500">

In [0]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [11]:
!git clone https://github.com/xlnwel/deep-learning.git
%cd deep-learning/tensorboard
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

fatal: destination path 'deep-learning' already exists and is not an empty directory.
/content/deep-learning/tensorboard/deep-learning/tensorboard


Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [12]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [13]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70],
      dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [14]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="https://github.com/xlnwel/deep-learning/blob/master/intro-to-rnns/assets/sequence_batching@1x.png?raw=1" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [0]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[: n_batches * characters_per_batch]
    
    # Reshape into batch_size rows
    arr = np.reshape(arr, (batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n: n+n_steps]
        # The targets, shifted by one
        y = np.concatenate((x[:, 1:], x[:, 0:1]), axis=1)
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [0]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [17]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="https://github.com/xlnwel/deep-learning/blob/master/intro-to-rnns/assets/charRNN.png?raw=1" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [0]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, (batch_size, num_steps))
    targets = tf.placeholder(tf.int32, (batch_size, num_steps))
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32)
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [0]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
        # Add dropout to the cell outputs
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob)
                                       for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [0]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    x = tf.reshape(lstm_output, (-1, in_size))
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size)) / tf.sqrt(tf.cast(in_size, tf.float32)))
        softmax_b = tf.zeros(out_size)
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.add(tf.matmul(x, softmax_w), softmax_b)
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits)
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [0]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, (-1, num_classes))
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_reshaped, logits=logits)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [0]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [0]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        with tf.device('/gpu:0'):
            # Build the input placeholder tensors
            self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

            # Build the LSTM cell
            cell, self.initial_state = build_lstm(lstm_size, num_layers,
                                                  batch_size, self.keep_prob)

            ### Run the data through the RNN layers
            # First, one-hot encode the input tokens
            x_one_hot = tf.one_hot(self.inputs, num_classes)

            # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
            outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
            self.final_state = state

            # Get softmax predictions and logits
            self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)

            # Loss and optimizer (with gradient clipping)
            self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
            self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [0]:
batch_size = 10         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [25]:
epochs = 20
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 50...  Training loss: 3.1441...  0.1231 sec/batch
Epoch: 1/20...  Training Step: 100...  Training loss: 2.7124...  0.1178 sec/batch
Epoch: 1/20...  Training Step: 150...  Training loss: 2.4858...  0.1235 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 2.5533...  0.1198 sec/batch
Epoch: 1/20...  Training Step: 250...  Training loss: 2.4847...  0.1230 sec/batch
Epoch: 1/20...  Training Step: 300...  Training loss: 2.3637...  0.1205 sec/batch
Epoch: 1/20...  Training Step: 350...  Training loss: 2.2563...  0.1206 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.2782...  0.1220 sec/batch
Epoch: 1/20...  Training Step: 450...  Training loss: 2.3077...  0.1210 sec/batch
Epoch: 1/20...  Training Step: 500...  Training loss: 2.3163...  0.1173 sec/batch
Epoch: 1/20...  Training Step: 550...  Training loss: 2.1052...  0.1186 sec/batch
Epoch: 1/20...  Training Step: 600...  Training loss: 1.9990...  0.1218 sec/batch
Epoch: 1/20...  T

Epoch: 1/20...  Training Step: 2450...  Training loss: 2.1450...  0.1383 sec/batch
Epoch: 1/20...  Training Step: 2500...  Training loss: 1.8971...  0.1251 sec/batch
Epoch: 1/20...  Training Step: 2550...  Training loss: 1.9000...  0.1263 sec/batch
Epoch: 1/20...  Training Step: 2600...  Training loss: 1.8352...  0.1200 sec/batch
Epoch: 1/20...  Training Step: 2650...  Training loss: 2.0271...  0.1212 sec/batch
Epoch: 1/20...  Training Step: 2700...  Training loss: 1.9786...  0.1199 sec/batch
Epoch: 1/20...  Training Step: 2750...  Training loss: 1.7905...  0.1210 sec/batch
Epoch: 1/20...  Training Step: 2800...  Training loss: 1.7400...  0.1215 sec/batch
Epoch: 1/20...  Training Step: 2850...  Training loss: 2.0168...  0.1268 sec/batch
Epoch: 1/20...  Training Step: 2900...  Training loss: 1.7632...  0.1259 sec/batch
Epoch: 1/20...  Training Step: 2950...  Training loss: 1.7322...  0.1187 sec/batch
Epoch: 1/20...  Training Step: 3000...  Training loss: 1.8596...  0.1208 sec/batch
Epoc

Epoch: 2/20...  Training Step: 4850...  Training loss: 1.7548...  0.1201 sec/batch
Epoch: 2/20...  Training Step: 4900...  Training loss: 1.8312...  0.1188 sec/batch
Epoch: 2/20...  Training Step: 4950...  Training loss: 1.7307...  0.1219 sec/batch
Epoch: 2/20...  Training Step: 5000...  Training loss: 1.7367...  0.1249 sec/batch
Epoch: 2/20...  Training Step: 5050...  Training loss: 1.6643...  0.1218 sec/batch
Epoch: 2/20...  Training Step: 5100...  Training loss: 1.6557...  0.1216 sec/batch
Epoch: 2/20...  Training Step: 5150...  Training loss: 1.8221...  0.1227 sec/batch
Epoch: 2/20...  Training Step: 5200...  Training loss: 1.8649...  0.1354 sec/batch
Epoch: 2/20...  Training Step: 5250...  Training loss: 1.6865...  0.1410 sec/batch
Epoch: 2/20...  Training Step: 5300...  Training loss: 1.8355...  0.1364 sec/batch
Epoch: 2/20...  Training Step: 5350...  Training loss: 1.8189...  0.1327 sec/batch
Epoch: 2/20...  Training Step: 5400...  Training loss: 1.8464...  0.1220 sec/batch
Epoc

Epoch: 2/20...  Training Step: 7250...  Training loss: 1.6649...  0.1225 sec/batch
Epoch: 2/20...  Training Step: 7300...  Training loss: 1.8345...  0.1218 sec/batch
Epoch: 2/20...  Training Step: 7350...  Training loss: 1.6648...  0.1175 sec/batch
Epoch: 2/20...  Training Step: 7400...  Training loss: 1.7020...  0.1234 sec/batch
Epoch: 2/20...  Training Step: 7450...  Training loss: 1.5829...  0.1203 sec/batch
Epoch: 2/20...  Training Step: 7500...  Training loss: 1.6120...  0.1256 sec/batch
Epoch: 2/20...  Training Step: 7550...  Training loss: 1.7166...  0.1206 sec/batch
Epoch: 2/20...  Training Step: 7600...  Training loss: 1.7438...  0.1202 sec/batch
Epoch: 2/20...  Training Step: 7650...  Training loss: 1.9666...  0.1289 sec/batch
Epoch: 2/20...  Training Step: 7700...  Training loss: 1.9098...  0.1253 sec/batch
Epoch: 2/20...  Training Step: 7750...  Training loss: 1.9499...  0.1215 sec/batch
Epoch: 2/20...  Training Step: 7800...  Training loss: 1.9039...  0.1211 sec/batch
Epoc

Epoch: 3/20...  Training Step: 9650...  Training loss: 1.7456...  0.1280 sec/batch
Epoch: 3/20...  Training Step: 9700...  Training loss: 1.7107...  0.1253 sec/batch
Epoch: 3/20...  Training Step: 9750...  Training loss: 1.5192...  0.1245 sec/batch
Epoch: 3/20...  Training Step: 9800...  Training loss: 1.7405...  0.1372 sec/batch
Epoch: 3/20...  Training Step: 9850...  Training loss: 1.7060...  0.1231 sec/batch
Epoch: 3/20...  Training Step: 9900...  Training loss: 1.7417...  0.1244 sec/batch
Epoch: 3/20...  Training Step: 9950...  Training loss: 1.8218...  0.1222 sec/batch
Epoch: 3/20...  Training Step: 10000...  Training loss: 1.7019...  0.1233 sec/batch
Epoch: 3/20...  Training Step: 10050...  Training loss: 1.7092...  0.1232 sec/batch
Epoch: 3/20...  Training Step: 10100...  Training loss: 1.7409...  0.1246 sec/batch
Epoch: 3/20...  Training Step: 10150...  Training loss: 1.7836...  0.1231 sec/batch
Epoch: 3/20...  Training Step: 10200...  Training loss: 1.9492...  0.1264 sec/batch

...

Epoch: 18/20...  Training Step: 69650...  Training loss: 1.6814...  0.1253 sec/batch
Epoch: 18/20...  Training Step: 69700...  Training loss: 1.6856...  0.1217 sec/batch
Epoch: 18/20...  Training Step: 69750...  Training loss: 1.8031...  0.1188 sec/batch
Epoch: 18/20...  Training Step: 69800...  Training loss: 1.5837...  0.1229 sec/batch
Epoch: 18/20...  Training Step: 69850...  Training loss: 1.7069...  0.1233 sec/batch
Epoch: 18/20...  Training Step: 69900...  Training loss: 1.5196...  0.1230 sec/batch
Epoch: 18/20...  Training Step: 69950...  Training loss: 1.6928...  0.1230 sec/batch
Epoch: 18/20...  Training Step: 70000...  Training loss: 1.6374...  0.1223 sec/batch
Epoch: 18/20...  Training Step: 70050...  Training loss: 1.4611...  0.1352 sec/batch
Epoch: 18/20...  Training Step: 70100...  Training loss: 1.6676...  0.1171 sec/batch
Epoch: 18/20...  Training Step: 70150...  Training loss: 1.6808...  0.1234 sec/batch
Epoch: 18/20...  Training Step: 70200...  Training loss: 1.6352..

Epoch: 19/20...  Training Step: 72050...  Training loss: 1.7303...  0.1405 sec/batch
Epoch: 19/20...  Training Step: 72100...  Training loss: 1.4751...  0.1395 sec/batch
Epoch: 19/20...  Training Step: 72150...  Training loss: 1.5457...  0.1417 sec/batch
Epoch: 19/20...  Training Step: 72200...  Training loss: 1.5877...  0.1374 sec/batch
Epoch: 19/20...  Training Step: 72250...  Training loss: 1.5651...  0.1415 sec/batch
Epoch: 19/20...  Training Step: 72300...  Training loss: 1.5076...  0.1280 sec/batch
Epoch: 19/20...  Training Step: 72350...  Training loss: 1.5935...  0.1265 sec/batch
Epoch: 19/20...  Training Step: 72400...  Training loss: 1.5497...  0.1410 sec/batch
Epoch: 19/20...  Training Step: 72450...  Training loss: 1.4967...  0.1233 sec/batch
Epoch: 19/20...  Training Step: 72500...  Training loss: 1.6181...  0.1211 sec/batch
Epoch: 19/20...  Training Step: 72550...  Training loss: 1.8034...  0.1240 sec/batch
Epoch: 19/20...  Training Step: 72600...  Training loss: 1.7199..

Epoch: 19/20...  Training Step: 74450...  Training loss: 1.6544...  0.1198 sec/batch
Epoch: 19/20...  Training Step: 74500...  Training loss: 1.7084...  0.1198 sec/batch
Epoch: 19/20...  Training Step: 74550...  Training loss: 1.6155...  0.1198 sec/batch
Epoch: 19/20...  Training Step: 74600...  Training loss: 1.7240...  0.1277 sec/batch
Epoch: 19/20...  Training Step: 74650...  Training loss: 1.5662...  0.1272 sec/batch
Epoch: 19/20...  Training Step: 74700...  Training loss: 1.5288...  0.1193 sec/batch
Epoch: 19/20...  Training Step: 74750...  Training loss: 1.7694...  0.1227 sec/batch
Epoch: 19/20...  Training Step: 74800...  Training loss: 1.5789...  0.1250 sec/batch
Epoch: 19/20...  Training Step: 74850...  Training loss: 1.5438...  0.1220 sec/batch
Epoch: 19/20...  Training Step: 74900...  Training loss: 1.6748...  0.1180 sec/batch
Epoch: 19/20...  Training Step: 74950...  Training loss: 1.7199...  0.1205 sec/batch
Epoch: 19/20...  Training Step: 75000...  Training loss: 1.6208..

Epoch: 20/20...  Training Step: 76850...  Training loss: 1.6837...  0.1214 sec/batch
Epoch: 20/20...  Training Step: 76900...  Training loss: 1.4951...  0.1168 sec/batch
Epoch: 20/20...  Training Step: 76950...  Training loss: 1.6026...  0.1217 sec/batch
Epoch: 20/20...  Training Step: 77000...  Training loss: 1.6407...  0.1243 sec/batch
Epoch: 20/20...  Training Step: 77050...  Training loss: 1.7535...  0.1205 sec/batch
Epoch: 20/20...  Training Step: 77100...  Training loss: 1.7052...  0.1234 sec/batch
Epoch: 20/20...  Training Step: 77150...  Training loss: 1.5842...  0.1250 sec/batch
Epoch: 20/20...  Training Step: 77200...  Training loss: 1.7319...  0.1216 sec/batch
Epoch: 20/20...  Training Step: 77250...  Training loss: 1.5920...  0.1226 sec/batch
Epoch: 20/20...  Training Step: 77300...  Training loss: 1.5004...  0.1222 sec/batch
Epoch: 20/20...  Training Step: 77350...  Training loss: 1.6361...  0.1227 sec/batch
Epoch: 20/20...  Training Step: 77400...  Training loss: 1.5826..

Epoch: 20/20...  Training Step: 79250...  Training loss: 2.1817...  0.1216 sec/batch
Epoch: 20/20...  Training Step: 79300...  Training loss: 3.6634...  0.1177 sec/batch
Epoch: 20/20...  Training Step: 79350...  Training loss: 3.9697...  0.1235 sec/batch
Epoch: 20/20...  Training Step: 79400...  Training loss: 5.7141...  0.1208 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [26]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i79400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62600_l128.ckpt"
al

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [0]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [0]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [29]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i79400_l128.ckpt'

In [30]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i79400_l128.ckpt
Faric spovllaman wammer.

"Th surkther te'tluuterhergys**3**3E.EE2*3ENI*IN*IE*E"S N.*N.NN.IINE.IRNIN*.*CNNEI**N.ENNI.II*.EEIN.IN*ENN.LL..IEN.E.E .EI.I.NEINE.EE.ELILEE.LEIE.EEIN.INNLNIL.
EINE.INIENL"E.NEL.  L..NIENIE.N.IINIENINIII.NENIN.IN.LI.E NNI L.N.I.E1NNE..NENIL.IIEENI..NNI.IIoF LNNIL.INI.E
 MEIEE.LENNII.E.NLIIN.ENL. ..E..NIILI"NIIL.INL.IL..IEN.E.EIEI.EELNILENENE.NNLNEEIL.IL.NEI.LINLLIII.E EIILM...NELNLININLNIILIILINILLINNNIL.ENEELEENNELNoIIN NINNIENNEI..ELLEEI.IE.LI NLNNELCEIIE.NN..ELN.IINL.INLE".ELNII E1IL..LN.E.E NLLNI...IEELIE.N.EN.
E. I .NEN.LINNI.ILLE.NN.NIIE.E.L..NLEL.IE.. LNIIENEN .INNLEELLEIL.EEILL..INEELINRENEIIL.NNNLENEENLE...N.LEL."E NE.NILLENE..L.N.NLIEII.EILLLI.IIN...E.ENENLNEEEIELLILMI...I.NINENNNEILE..NN..I EE"MvLIELELLELNENIEE..LNEEEL..NNI..NN.L. .LNNMIE
.  EINIEN.E CNLL. I.INNE.I.I.EIN IENEN.NLI N
.LEE. LN.ENNINEN. NENE.INIELL*EE.EI LINE.LLNNELEIUNL NI.LI"N"LEIN..EI."IEIIEMLNI..IEEILLIENNNNL..N

In [33]:
checkpoint = 'checkpoints/i78000_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i78000_l128.ckpt
Fares, and he should be that he was taken always to hear and at all
house, and as at onces and and sayings
on her trace and that the
white children had not and sat down to win himself. He was
in the mone the prince.

"Would I say is this time. What would be shame of
the
sister, why would anyone. I was that they as you was all had told her and then in home, and it is
as it is and husbands was the white cheal instood the way there seemed. I am
thought that I dead. I've seen the moment to think," she said and whole sound and a met if he was not to say
to the states to way, standing of to both to having of it
that
an a taking, and he asked the selfice of a peason that throat his at the
stird of a comploinges, hearing his seaser the servood, when the considers to a sours was a son to
her well and all the position of his head
to want his wend as he was a month of the same thong, and
which she can allot watch, astile the
m

In [34]:
checkpoint = 'checkpoints/i61800_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i61800_l128.ckpt
Fardina and he had song of the pressing the convinceds of the cold, something with a man
was satisfaction he was to the pissing his foingousion where he story the
call on the pirte was all there and step and happy had said to
teel, him that he would be
too
on
in the peast worth and the shamod
and took to his hands.

"It's better things and a sumping and so her hand and that I can't see the same?"

"What I said.
I've a sunder to me her same
to his to the same set of him at her the
was they seemed to her than
an ordination in a cangance, I wanted on a tears of the politice with the till to home. That with the
their way, there's the man seed in hand in the preat of that had to think in me to
said to still wouldn't let a streater. I'm set the same," and hopes he carryed at happine, would his there and as though she something he would necesse house
any thought.

Stepan Arkadyevitch, so he should be said to that a settang

In [35]:
checkpoint = 'checkpoints/i67000_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i67000_l128.ckpt
Farsate and saw ordering.
"I so thought of the commans tree of this
thing, and I helvend him to too over a
carriage all it?" said Vronsky.

"Alexey
I can't talk with minge of and distelles. His so most," she said. "Why to see that all
he was so
to the prince. I say what he went to her sellenteled their, and I wing the point of their time of at immerity,
with most, but it had to be, think to and and attersions, I did not boring thought to mut her to sort in her as you then, I were the drink in the picture," said Stepan Arkadyevitch's house.

She was satisfactation,
they had at them, to say, when the shere he was not that his
to her at her, a strility and to this
shoulder alone as it was before. The pole hands there was a continial chatmer
and at it in to them, but it was the
sole on a peasants and would not said to the councitally that the same thing
of those what his son which seely, and the pathing on a mans.

"Was